# Connected Platform - Neptune Asset Management

In this workshop, you will first build an asset management platform using AWS Neptune. Then, you will query the platform to gain insightful relationships that can drive additional sales revenue.

The following fictatious scenario is modeled in AWS Neptune:

John is a scientist and an employee of ACME Lab Corporation .  John has purchased an Ultra-Low Temperature (ULT) freezer, DNA Sequencer (QuantStudio), and a master mix regent to perform his experiments.  As John works with these instruments, his freezer door malfunctions.  ACME corporation has a ‘silver’ tier contract and notifies the manufacturer that the freezer door has malfunctioned.  The manufacturer of the freezer assigns a service ticket to Joe - a knowledgeable service technician. 
  

The fictatious scenario described can be modeled in AWS Neptune:

![Connected platform](https://serversless-immersion-api-gateway.s3.amazonaws.com/assetmangement.png)

## Driving Sales Revenue 
Once the fictatious scenario above is modeled, an analyst can drive additional sales revenue by asking neptune the following quetions:
1. Who are the top customers that would benefit from a ‘gold’ service contract?  
2. What other forensic labs can we target to purchase a DNA Sequencer ?

# Part 1: Building an Asset Management Platform

**Pre-requisites**
This workshop assumes the following:
1. you have successfully created a neptune cluster
2. You are running this notebook from a connected neptune workbench.

As a first step, let's connect to our neptune cluster and make sure the stauts is healthy.

In [1]:
%status

{'status': 'healthy',
 'startTime': 'Wed Sep 23 13:51:24 UTC 2020',
 'dbEngineVersion': '1.0.3.0.R1',
 'role': 'writer',
 'gremlin': {'version': 'tinkerpop-3.4.3'},
 'sparql': {'version': 'sparql-1.1'},
 'labMode': {'ObjectIndex': 'disabled',
  'DFEQueryEngine': 'disabled',
  'ReadWriteConflictDetection': 'enabled'}}

## Adding Vertices and Edges in Neptune
In AWS Neptune, you can use Gremline or SPARQL.  In this workshop, you will build the asset management using Gremline. 

In Gremline, a graph is a structure composed of vertices and edges. Both vertices and edges can have an arbitrary number of key/value-pairs called properties.  Vertice are nodes that model a specific object, such as a person, place or event.  Edges are the relationships that exists between the vertices in a graph.

The following sections add Vertices and Edges to the neptune graph

### Adding Vertices
First, we will add all the vertices that model sale and service entities. 

1. **Add ACME Company Vertex**
<br/> Start by adding ACME corporation vertex.
<br/> ACME is of **type** organization and has a vertex id of 1 and a name "ACME Corporation"

In [2]:
%%gremlin

g.addV('organization').property(id, '1').property('name', 'ACME Corporation')

2. **Add customer vertex with properties** 
<br/>John  is a scientist who is a customer.  John's location and name is captured as properties.  

In [3]:
%%gremlin

g.addV('customer').property(id, '2').property('name', 'John').property('location','123 ACME Drive. Los Angeles, CA 90210')

3. **Add all other vertexes.**
<br/> The following vertecies are added with their associated properties
<br/> Joe - Service Technican.  Properties include service review score and physical location.
<br/> QuantStudio - lab instrument
<br/> Master Mix - regent used for expiermentation
<br/> Freezer - labratory freezer
<br/> V-Drive - Freezer part
<br/> ticket - service ticket to repair freezer
<br/> Contract - service contract for ACME corporation.


In [4]:
%%gremlin
g.addV('technician').property(id, '3').property('name', 'Joe').property('score', '4').property('location', 'Los Angeles').next()
g.addV('device').property(id, '4').property('name', 'QuantStudio').property('status', 'online').property('defectCount', '0').next()
g.addV('consumable').property(id, '5').property('name', 'MasterMix').property('orderdate', '01/06/2020').property('count', '4').next()
g.addV('device').property(id, '6').property('name', 'Freezer').property('status', 'online').property('defectCount', '0').next()
g.addV('part').property(id, '7').property('name', 'VDrive').property('status', 'online').property('type', 'nema 5-20').next()
g.addV('contract').property(id, '8').property('name', 'service contract').property('type', 'silver').next()
g.addV('workorder').property(id, '9').property('name', 'ticket')

### Adding Edges

Next we will add the edges that represent the relationships between sale and service data <br/>

**Create bi-directional relationship (edges) between John and ACME.**
<br/>Since John is an employee of ACME, we can create an edge to depict that John works at ACME.  In the same manner, ACME employs John.  

In [5]:
%%gremlin

g.V('1').addE('employs').to(g.V('2')).next()
g.V('2').addE('worksAt').to(g.V('1'))


Let's add all the other relationships:

5. **Adding relationships**
<br/> ACME corporation has a service contract
<br/> John has purchased a QuantStudio, Master Mixm, and freezer
<br/> Master Mix is a regent of QuantStudio
<br/> The freezer has a part - VDrive 

In [6]:
%%gremlin

g.V('1').addE('support').to(g.V('8')).next()
g.V('2').addE('purchased').to(g.V('4')).next()
g.V('2').addE('purchased').to(g.V('5')).next()
g.V('2').addE('purchased').to(g.V('6')).next()
g.V('5').addE('regent').to(g.V('4')).next()
g.V('6').addE('part').to(g.V('7')).next()

### Updating Neptune

Now, let's assume that the VDrive part of the freezer has failed.  We need to change the status of the **freezer** to offline and increment the defect count.

In [7]:
%%gremlin
g.V('6').property(single, 'status', 'offline').next()
g.V('6').property(single, 'defectCount', '4')

In [8]:
%%gremlin
g.V('9').addE('assigned').to(g.V('3')).next()
g.V('3').addE('fix').to(g.V('6'))

# Performing queries as an Analyst

Now that the asset management platform has been constructed, an analysts can run the following queries to gain insight and drive additional revenue for ACME corporation:

**1. Who are the top customers that would benefit from a ‘gold’ service contract?**

 To answer this quesiton, we want to traverse the graph and get all the 'silver' customers who have purchased a product with more than 3 defects.  Customers who have lots of defects should upgrade to 'Gold' status to get faster service.

In [9]:
%%gremlin
g.V().hasLabel('contract', 'type', 'silver').inE('support').outV().outE('employs').inV()
.hasLabel('customer').out('purchased').values('defectCount').is(gt('1'))

In [10]:
%%gremlin
g.V().hasLabel('contract', 'type', 'silver').inE('support').outV().outE('employs').inV()
.hasLabel('customer').out('purchased').values('defectCount').is(gt('1')).path()

In [11]:
%%gremlin
g.V().hasLabel('contract', 'type', 'silver').inE('support').outV().as('company').outE('employs').inV()
.hasLabel('customer').out('purchased').values('defectCount').is(gt('1')).path().from('company').to('company').by('name')

The result above shows that ACME corporation  can benefit from upgading their silver contract to gold.  The upgarde to gold can result in additional revenue !

**2. Challenge Question: What other forensic labs can we target to purchase a DNA Sequencer ?**

Hint: <br/>
<br/>(1) Create additional vertices - ABC Lab and Forensic Labs
<br/>(2) Create Edge between ABC Lab and its parent company Forensic Labs
<br/>(3) Establish an edge between Forensic Lab and ACME corporation
<br/>(4) Write the Gremline query to retrieve all 'organizations' that have not purchased a QuantStudio - start at Forensic Lab vertex.


In [17]:
%%gremlin
g.V().groupCount().by(T.label).toList()


In [18]:
%%gremlin
g.E().groupCount().by(T.label).toList()

# Cleanup
**TO delete your graph, run the following command**
<br/>Note: This does not stop/terminate your neptune cluster.  You will have to stop/terminate your cluster from AWS neptune console or AWS CLI.

In [ ]:
%%gremlin
g.V().drop().iterate()